<a href="https://colab.research.google.com/github/s528661/MyBERT_Practice/blob/main/BERT_NextSentencePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install folium==0.2.1
!pip install urllib3==1.25.11
!pip install pytorch-transformers==1.2.0
!pip install fugashi
!pip install ipadic

In [34]:
import torch
from transformers import BertJapaneseTokenizer, BertForNextSentencePrediction
from pytorch_transformers import BertTokenizer

In [35]:
import pandas as pd

In [36]:
data = pd.read_parquet("hf://datasets/fujiki/llm-japanese-dataset_wikinews/data/train-00000-of-00001-b009aac0a604af5e.parquet")

In [44]:
# sample 1 (Real Next Sentences)
sentences = data.iloc[0,:]['input'].split('。')
text1 = '[CLS] ' + sentences[0] + '。 [SEP] ' + sentences[1] + '。 [SEP]'
print(text1)

[CLS] 台湾の自由時報によると、3日、陳水扁総統夫人の呉淑珍氏と馬永成元総統府副秘書長、林徳訓総統府秘書らが国務機密費の流用容疑で、台北地方検察院により起訴された。 [SEP] 台北地検は、総統夫人を主犯、総統府秘書であった馬氏、林氏の両名が共犯と認定した。 [SEP]


In [45]:
# sample 2 (Fake Next Sentences)
text2 = '[CLS] ' + data.iloc[1,:]['input'].split('。')[0] + '。 [SEP] '  + data.iloc[2,:]['input'].split('。')[0] + '。 [SEP]'
print(text2)

[CLS] 岩波書店は約10年ぶりの大改訂となる「広辞苑 第六版」を1月11日に発売した。 [SEP] 4日、長野県安曇野市の長野県立こども病院は、心臓疾患を持って1,100gの低出生体重で生まれた男児の心臓手術に成功したと発表した。 [SEP]


In [39]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [46]:
def show_continuity(text, seg_ids):
    words = tokenizer.tokenize(text)
    word_ids = tokenizer.convert_tokens_to_ids(words)  # word2index
    word_tensor = torch.tensor([word_ids])  # index2tensor

    seg_tensor = torch.tensor([seg_ids])

    nsp_model = BertForNextSentencePrediction.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
    nsp_model.cuda()
    nsp_model.eval()

    X = word_tensor.cuda()
    s = seg_tensor.cuda()

    y = nsp_model(X, s)  # 予測
    result = torch.softmax(y[0], dim=1)
    print(result)  # Softmaxで確率に
    print(str(result[0][0].item()*100) + "%の確率で連続しています。")

In [64]:
def seg_ids_create(text):
    seg_ids = [0] * len(tokenizer.tokenize(text))  # 0:前の文章の単語、1:後の文章の単語

    sep_cnt = 0
    for idx, word in enumerate(tokenizer.tokenize(text)):
        seg_ids[idx] += sep_cnt
        if word == '[SEP]':
            sep_cnt += 1

    return seg_ids

In [65]:
# sample 1 (Real Next Sentences)
seg_ids1 = seg_ids_create(text1)
print(seg_ids1)
show_continuity(text1, seg_ids1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[9.9975e-01, 2.4774e-04]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
99.97522234916687%の確率で連続しています。


In [66]:
# sample 2 (Fake Next Sentences)
seg_ids2 = seg_ids_create(text2)
print(seg_ids2)
show_continuity(text2, seg_ids2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[0.9943, 0.0057]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
99.4261384010315%の確率で連続しています。
